# Algoritam 

% Ulazi: $H, v, w, \alpha$ $\quad$ Izlaz: $\hat{\pi}$

% Metoda potencija primijenjena na $G^{(1)}$:

Odaberi početni vektor $\hat{\sigma}^{T} = [\hat{\sigma}^{T}_{1:k} \quad \hat{\sigma}_{k + 1}]$ sa $\hat{\sigma} \ge 0$, $||\hat{\sigma}|| = 1$.

$\textbf{Dok}$ nije konvergiralo:

$\qquad \hat{\sigma}^{T}_{1:k} = \alpha \hat{\sigma}^{T}_{1:k} H_{11} + (1-\alpha)v_1^T + \alpha \hat{\sigma}_{k + 1} w_1^T$

$\qquad \hat{\sigma}_{k + 1} = 1 - \hat{\sigma}^{T}_{1:k} e$

$\textbf{zavrsi dok}$

% Vrati PageRank

$\hat{\pi}^T = [\hat{\sigma}^{T}_{1:k} \quad \alpha \hat{\sigma}^{T}_{1:k} H_{12} + (1-\alpha)v_2^T + \alpha \hat{\sigma}_{k + 1} w_2^T]$ 

In [353]:
import numpy as np
from copy import deepcopy

In [312]:
def get_H_11_and_H_12(H):
    k = -1
    for i in range(len(H)):
        if not H[i, :].any():
            k = i
            break

    if k == -1:
        return "Error"
    
    H_11 = H[0:k, 0:k]

    H_12 = H[:len(H)-k+1, k:]
    
    return H_11, H_12

In [345]:
def pagerank(H, v, w, a, conv = 0.01):
    H_11, H_12 = get_H_11_and_H_12(H)
    k = len(H_11)
    
    w_1_T = w[:k].transpose()
    w_2_T = w[k:].transpose()

    v_1_T = v[:k].transpose()
    v_2_T = v[k:].transpose()
    
    sigma = np.random.rand(k+1)
    sigma = sigma / sum(sigma)
    sigma_T = sigma.transpose()
    
    e = np.ones(k)
    
    err = float("inf")
    while err > conv:
        sigma_T_ = deepcopy(sigma_T)
        sigma_T_[:k] = a * sigma_T[:k] @ H_11 + (1 - a) * v_1_T + a * sigma_T[k] * w_1_T
        sigma_T_[k] = 1 - sigma_T_[:k] @ e
        err = sum(abs(sigma_T_ - sigma_T))
        sigma_T = deepcopy(sigma_T_)

    return np.concatenate((sigma_T_[:k], a * sigma_T_[:k] @ H_12 + (1 - a) * v_2_T + a * sigma_T[k] * w_2_T), axis=None)

In [352]:
H = np.array([
    [0, 0, 1/2, 1/2, 0],
    [0, 0, 1/2, 1/2, 0],
    [1/3, 1/3, 0, 1/3, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])


w = np.array([
    [0],
    [0],
    [0],
    [1],
    [1],
])

w = w / sum(w)


v = np.array([
    [3],
    [2],
    [2],
    [1],
    [1],
])

v = v / sum(v)


pi = pagerank(H, v, w, 0.5)

pi

array([0.19963539, 0.14407984, 0.19755655, 0.28909251, 0.17023761])